> scaling of (Generalist) reward model

- 两个重点
    - 一个是 **scaling**
        - from one to three scaling laws
            - pre-training
            - post-training
                - RL
            - test-time scaling
                - 根据 query，实时对计算弹性伸缩；
    - 一个是 **general** (通用性)

### Point-wise GRM

- principle => critic => score
    - 奖励过程思维链化（CoT）了
    - 定标准，写评价，再打分；

### 输入输出的角度

- GRM 的输入与传统的奖励模型非常相似，主要包括：
    - 查询 (Query)：用户的提问或指令，在论文中用 $x$ 表示。
    - 一个或多个候选回答 (Responses)：模型针对该查询生成的不同回答，在论文中用 $\{y_i\}$ 表示。
    - 这篇论文的一个创新点在于，他们还引入了一个额外的输入：
        - 原则 (Principles)：这是一组预先定义或由模型自己生成的评估标准/指导原则，在论文中用 $\{p_j\}$ 表示。这些原则告诉模型应该从哪些维度（如准确性、清晰度、安全性等）来评估回答。
- 所以，GRM 的完整输入可以看作是：(查询, 候选回答, 评估原则)。
    - $r_θ(x, {y_i}, {p_j})$

- 这是 GRM 与传统标量奖励模型最根本的区别。GRM 的输出是一段结构化的自然语言文本，这篇论文中称之为**“评判” (Critique)**，用 $C$ 表示。
    - 分析过程 (Analysis)：模型会像一个真正的裁判一样，根据输入的“评估原则”，逐一分析每个“候选回答”的优缺点。
    - 对比和结论 (Comparison & Conclusion)：在分析之后，模型会给出一个明确的对比结论，比如“回答2比回答1更好”。
    - 最终的奖励分数 (Final Scores)：最关键的是，在这段生成的文本末尾，模型会按照一个预设的格式输出最终的打分。

### Meta RM

- 第二层级的、独立的奖励模型，即元奖励模型 (Meta Reward Model, Meta RM) 输出

### Voting

- table 17
    - 模型单次运行的结果可能存在随机性或偏差。为了得到一个更稳定、更可靠的评估，研究人员会让同一个模型（DeepSeek-GRM）对同样的问题和回答，独立地运行多次（在这个表格里是3次，产生了 Result 1, 2, 3）。
    - Result 1 的打分是 \boxed{8, 8}。
        - Response 1 得了 8 分。
        - Response 2 得了 8 分。
    - Result 2 的打分是 \boxed{9, 5}。
        - Response 1 得了 9 分。
        - Response 2 得了 5 分。
    - Result 3 的打分是 \boxed{10, 7}。
        - Response 1 得了 10 分。
        - Response 2 得了 7 分。
    - Response 1 的总票数: 8 (来自Result 1) + 9 (来自Result 2) + 10 (来自Result 3) = 27 分
    - Response 2 的总票数: 8 (来自Result 1) + 5 (来自Result 2) + 7 (来自Result 3) = 20 分
- The input order of responses is reversed for DeepSeek-GRM-27B when generating result 2 and result 3."
    - 这意味着在第2次和第3次运行时，模型看到的“Response 1”其实是我们的“Response 2”，它看到的“Response 2”其实是我们的“Response 1”。这是一种为了避免位置偏见而常用的技巧。